# Notes

content --> senses --> index --> grammar contains verb info

In [2]:
import pymongo
import json
import nltk

In [3]:
pswrd = ''
with open('pswrd.txt') as text: pswrd = text.read()
conn_str = 'mongodb+srv://dov-db2:' + pswrd + '@apicluster.s8lqy.mongodb.net/test'
_client = pymongo.MongoClient(conn_str)
_db = _client['bavli']
jastrow = _db['dov-jastrow']
dicta_all = _db['new-dicta']

In [3]:
items = list(jastrow.find())

# POS Tagger

In [4]:
from utils.get_pos import *

In [26]:
w = list( jastrow.aggregate([{ '$sample': { 'size': 1 } }]) )[0]
no_html = remove_html(w)
no_html

{'headword': 'מְרוּמֶּה',
 'rid': 'M02593',
 'refs': ['Sanhedrin 32b:3', 'Sanhedrin 32b:4', 'Shevuot 30b:15'],
 'content': {'senses': [{'definition': ' ( רָמָה )  fraudulent ;  דין מ׳  a case in which the court has reason to suspect legal trickery or conspiracy.  Snh. 32ᵇ .  Shebu. 30ᵇ '}],
  'morphology': 'm.'},
 'plural_form': [],
 'alt_headwords': [],
 'unvoweled': 'מרומה',
 'word': 'מְרוּמֶּה',
 'more_forms': [],
 'all_forms': ['מְרוּמֶּה'],
 'all_unvoweled': ['מרומה'],
 '_id': ObjectId('60ca636a83050705044c939f')}

In [27]:
flat = flatten_dict(no_html)
flat

{'headword': ['מְרוּמֶּה'],
 'rid': ['M02593'],
 'refs': ['Sanhedrin 32b:3', 'Sanhedrin 32b:4', 'Shevuot 30b:15'],
 'definition': [' ( רָמָה )  fraudulent ;  דין מ׳  a case in which the court has reason to suspect legal trickery or conspiracy.  Snh. 32ᵇ .  Shebu. 30ᵇ '],
 'morphology': ['m.'],
 'unvoweled': ['מרומה'],
 'word': ['מְרוּמֶּה'],
 'all_forms': ['מְרוּמֶּה'],
 'all_unvoweled': ['מרומה'],
 '_id': [ObjectId('60ca636a83050705044c939f')]}

In [28]:
all_defs = get_defs(flat['definition'])
all_defs

[('fraudulent',
  'a case in which the court has reason to suspect legal trickery or conspiracy')]

In [29]:
for defs in all_defs:
    for d in defs:
        if d == '': continue
        print(nltk.pos_tag(nltk.word_tokenize(d)))

[('fraudulent', 'NN')]
[('a', 'DT'), ('case', 'NN'), ('in', 'IN'), ('which', 'WDT'), ('the', 'DT'), ('court', 'NN'), ('has', 'VBZ'), ('reason', 'NN'), ('to', 'TO'), ('suspect', 'VB'), ('legal', 'JJ'), ('trickery', 'NN'), ('or', 'CC'), ('conspiracy', 'NN')]


In [11]:
tag_def(all_defs[0])

('DEF', 'NN')

# Percentage of words in Berakhot present in Dicta

In [4]:
from utils.texts import Masekhet
from utils.deconstruct2 import detach_prefix

In [5]:
ber = Masekhet('Berakhot')

In [6]:
gemara_only = []
for p in range(len(ber)):
    for c in range(len(ber[p])):
        if ber.tags[p][c] == 'm' or ber.tags[p][c] == 'mc':
            continue
        gemara_only += [t for t in ber[p][c] if t not in ',.?!\'\"״-—:;׳()[]']

In [14]:
num_words = len(gemara_only[:100])
in_db = 0
for t in gemara_only[:100]:
    ws = detach_prefix(t)
    print(ws, end='\t')
    in_db += any([dicta_all.count_documents({'word': w+'\t'}) for w in ws])
    print(in_db)
print(in_db / num_words)

['גְּמָ']	0
['תַּנָּא']	1
['הֵיכָא']	2
['קָאֵי', 'אֵי']	3
['דְּקָתָנֵי', 'תָנֵי', 'קָתָנֵי']	3
['מֵאֵימָתַי', 'אֵימָתַי']	3
['וְתוּ', 'תוּ']	3
['מַאי']	4
['שְׁנָא']	5
['דְּתָנֵי', 'תָנֵי']	5
['בְּעַרְבִית', 'עַרְבִית']	5
['בְּרֵישָׁא', 'רֵישָׁא']	6
['לִתְנֵי', 'תְנֵי']	7
['דְּשַׁחֲרִית', 'שַׁחֲרִית']	7
['בְּרֵישָׁא', 'רֵישָׁא']	8
['תַּנָּא']	9
['אַקְּרָא', 'קְּרָא']	10
['קָאֵי', 'אֵי']	11
['דִּכְתִיב', 'תִיב', 'כְתִיב']	11
['בְּשָׁכְבְּךָ', 'שָׁכְבְּךָ']	11
['וּבְקוּמֶךָ', 'קוּמֶךָ', 'בְקוּמֶךָ']	11
['וְהָכִי', 'י', 'הָכִי']	12
['קָתָנֵי', 'תָנֵי']	12
['זְמַן']	12
['קְרִיאַת']	12
['שְׁמַע']	13
['דִּשְׁכִיבָה', 'שְׁכִיבָה']	14
['אֵימַת']	15
['מִשָּׁעָה', 'שָּׁעָה']	15
['שֶׁהַכֹּהֲנִים', 'כֹּהֲנִים', 'הַכֹּהֲנִים']	15
['נִכְנָסִין']	15
['לֶאֱכוֹל']	15
['בִּתְרוּמָתָן', 'תְרוּמָתָן']	15
['וְאִי', 'אִי']	16
['בָּעֵית']	17
['אֵימָא']	18
['יָלֵיף']	19
['מִבְּרִיָּיתוֹ', 'רִיָּיתוֹ', 'בְּרִיָּיתוֹ']	19
['שֶׁל']	19
['עוֹלָם']	19
['דִּכְתִיב', 'תִיב', 'כְתִיב']	19
['וַיְהִי']	19
['עֶרֶב']	19
['

# Appearance of Tannaitic Corpus Words in Dicta

In [16]:
with open('data/rab_heb_words.txt', encoding='utf-8') as f:
    heb_words = f.read().split('\n')

In [ ]:
for w in heb_words:
    print(w, end='\t')
    print(dicta_all.count_documents({'word': w+'\t'}))

וַיֹּאמֶר	0
יי	0
אֶל	0
מֹשֶׁה	0
וְאֶל	0
אַהֲרֹן	0
בְּאֶרֶץ	0
מִצְרַיִם	0
לֵאמֹר	0
שׁוֹמֵעַ	0
אֲנִי	0
שֶׁהָיָה	0
הַדִּבֵּר	0
לְמֹשֶׁה	0
וּלְאַהֲרֹן	0
כְּשֶׁהוּא	0
אוֹמֵר	0
וַיְהִי	0
בְּיוֹם	0
דִּבֶּר	0
מִצְרָיִם	0
הָיָה	0
וְלֹא	0
לְאַהֲרֹן	0
אִם	1
כֵּן	0
מַה	0
תִּלְמֹד	0
לוֹמַר	1
אֶלָּא	0
מְלַמֵּד	0
שֶׁכַּשֵּׁם	0
כְּלָל	0
לַדִּבְּרוֹת	0
כָּךְ	0
וּמִפְּנֵי	0
מָה	1
לֹא	0
נִדְבַּר	0
עִמּוֹ	0
מִפְּנֵי	0
כְבוֹדוֹ	0
שֶׁלְּמֹשֶׁה	0
נִמְצֵאתָ	0
מְמָעֵט	0
מִכָּל	0
הַדִּבְּרוֹת	0
שֶׁבַּתּוֹרָה	0
חוּץ	5
מִשְּׁלֹשָׁה	0
מְקוֹמוֹת	0
שֶׁאֵי	0
אֶפְשָׁר	0
דָּבָר	0
אַחֵר	0
לָמָּה	0
נֶאֱמַר	0
לְפִי	0
שֶׁהוּא	0
רְאֵה	0
נְתַתִּיךָ	0
אֱלֹהִים	0
לְפַרְעֹה	0
אֵין	0
לִי	1
דַּיָּן	0
מְנַיִן	0
הִקִּישׁ	0
מֹּשֶׁה	0
אַף	1
דְּבָרָיו	0
יָרֵא	0
כָּל	0
הַקּוֹדֵם	0
בַּמִּקְרָא	0
הוּא	2
קוֹדֵם	0
בַּמַּעֲשֶׂה	0
וּמֹשֶׁה	0
מַגִּיד	0
שֶׁשְּׁנֵיהֶם	0
שְׁקוּלִין	0
זֶה	0
כַּזֶּה	0
כַּיּוֹצֵא	0
בַזֶּה	0
אַתָּה	0
בְּרֵאשִׁית	0
בָּרָא	0
אֵת	0
הַשָּׁמַיִם	0
וְאֵת	0
הָאָרֶץ	0
עֲשׂוֹת	0
אֶרֶץ	0
וְשָׁמָיִם	0
כְּאֶחָד	0
בַדָּבָר	0
אָנ

גְּדֹלוֹת	0
וּבְצֻרוֹת	0
יְדַעְתָּם	0
כֶּרֶם	0
סְיָג	0
צֹאן	0
רוֹעֶה	0
הִנְנִי	0
מֵבִיא	0
רָעָה	0
בָּשָׂר	0
וְנָתַתִּי	0
נַפְשְׁךָ	0
לְשָׁלָל	0
הַמְּקֹמוֹת	0
תֵּלֶךְ	0
מִתְנַּבְּאִים	0
בִּזְכוּתָן	0
שֶׁלְּיִשְׂרָאֵל	0
הַחֹדֶשׁ	0
הַזֶּה	0
רֹאשׁ	0
חֳדָשִׁים	0
רִאשׁוֹן	0
לְחָדְשֵׁי	0
הַשָּׁנָה	0
הֶרְאָה	0
לְיִשְׂרָאֵל	0
וְאָמַר	0
הֱיוּ	0
רוֹאִין	0
וְקוֹבְעִין	0
לַדּוֹרוֹת	0
דְבָרִים	0
שֶׁנִּתְקַשָּׁה	0
בָהֶן	0
וְהֶרְאָהוּ	0
כֻּלָּן	0
בָּאֶצְבַּע	0
הַטָּמֵא	0
מַעֲשֵׂה	0
הַמְּנֹרָה	0
בַּשְּׁחִיטָה	0
נִתְקַשָּׁה	0
תַּעֲשֶׂה	0
הַמִּזְבֵּחַ	0
יוֹחַי	0
בַיּוֹם	0
וְהַחֹדֶשׁ	0
הראהו	0
בַּלַּיְלָה	0
וְכֵיצַד	0
בַּיּוֹם	0
חֲשֵׁכָה	0
בַּחֲשֵׁכָה	0
נִיסָן	3
מֵחָדְשֵׁי	0
וְחַג	0
הָאָסִף	0
בְּצֵאת	0
הָאָסִיף	0
תְּקוּפַת	0
אֵי	0
חֹדֶשׁ	0
שֶׁיֵּשׁ	0
אָסִיף	0
וּתְקוּפָה	0
וְשָׁנָה	0
יוֹצְאָה	0
בוֹ	0
וְקָרוּי	0
שְׁבִיעִי	0
תִּשְׁרִי	0
לְאַחַר	1
שֶׁלָּמַדְתָּ	0
שֶׁשְּׁבִיעִי	0
רְאָיָה	0
לַדָּבָר	0
זֵכֶר	0
בַּחֹדֶשׁ	0
הָרִאשׁוֹן	0
שֶׁנִּיסָן	0
לֶחֳדָשִׁים	0
וּמְנַיִן	0
לַמְּלָכִים	0
הַשֵּׁנִי	0
לִמְלֹךְ	1
שְ

מִתְבַּשֵּׂר	0
בַּשּׁוּרָה	0
טוֹבָה	0
וְאֵינוֹ	0
שָׂמֵחַ	0
נוֹלַד	0
רַבָּךְ	0
מוֹצִיאָךְ	0
לְחֵרוּת	0
קָשֶׁה	0
בְּעֵינֵיהֶם	0
לִפְרֹשׁ	1
אֲלֵהֶם	0
שִׁקּוּצֵי	0
עֵינָיו	0
הַשְׁלִיכוּ	0
וּבְגִלּוּלֵי	0
תִּטַּמָּאוּ	0
וַיַּמְרוּ	0
בִי	0
אָבוּ	2
הִשְׁלִיכוּ	0
גִּלּוּלֵי	0
עָזָבוּ	0
לִשְׁפֹּךְ	0
חֲמָתִי	2
לְכַלּוֹת	0
אַפִּי	0
וָאַעַשׂ	0
לְבִלְתִּי	0
הֵחֵל	0
לְעֵינֵי	0
בְתוֹכָם	0
נוֹדַעְתִּי	0
לְעֵינֵיהֶם	0
לְהוֹצִיאָם	0
הָדָא	2
דִכְתִיב	0
וַיְצַוֵּם	0
צִוָּם	0
וַיִּקְרָא	0
לְכָל	0
זִקְנֵי	0
רַבֵּינוּ	0
נִזְבַּח	0
תּוֹעֲבַת	0
יִסְקְלֻנוּ	0
לָהֶן	0
הַנֵּס	0
בִּמְשִׁיכָתוֹ	0
אַתֶּם	0
יוֹדְעִים	0
בִּשְׁחִיטָתוֹ	0
שָׁמְרֵהוּ	0
וְשָׁחְטֵהוּ	0
וּמָשְׁכֵהוּ	0
בָּרִאשׁוֹן	0
קְבָעוֹ	0
חוֹבָה	0
מְבַקְּרִין	0
אוֹתוֹ	0
קֹדֶם	0
הַתָּמִיד	0
נֶאֶמְרָה	0
שְׁמִירָה	0
וְנֶאֶמְרָה	0
בַתָּמִיד	0
שְּׁמִירָה	0
הָאֲמוּרָה	0
שְׁחִיטָה	0
בַּתָּמִיד	0
פּוֹחֲתִים	0
מִשִּׁשָּׁה	0
טְלָאִים	0
הַמְּבֻקָּרִין	0
מִלִּשְׁכַּת	0
הַטְּלָאִים	0
לַשַּׁבָּת	0
וְלִשְׁנֵי	0
טוֹבִים	0
שֶׁלְּרֹאשׁ	0
וּמוֹסִיפִין	0
בַּחֹל	0
בַּשַּׁבָּ

חֲמֵשֶׁת	0
הַמִּינִין	0
וְאֵלּוּ	0
הַחִטִּים	0
וְהַשְּׂעוֹרִים	0
וְהַכֻּסְּמִים	0
וְשִׁבֹּלֶת	0
שׁוּעָל	0
וְשִׁיפוֹן	0
הָאֹרֶז	0
וְהַדֹּחַן	0
וְהַפְּרָגִין	0
וְהַקִּטְנִיּוֹת	0
וְהַשַּׁמְשְׁמִין	0
בָּאִין	0
סֵרָחוֹן	0
קְדֵרָה	0
בְמַשְׁמָע	0
לֶחֶם	0
הָאֶחָד	0
וְעֶשְׂרִים	0
אוֹצִיא	0
סָמוּךְ	0
לְעֶשְׂרִים	0
וּשְׁאָר	0
רְשׁוּת	0
בָּרִאשֹׁן	0
בָּעֶרֶב	0
שֵׁשֶׁת	0
בַּכְּלָל	0
מוּצָא	0
הַכְּלָל	0
שְּׁבִיעִי	0
בַכְּלָל	0
שִׁבְעָה	0
הֶחָדָשׁ	0
הַיָּשָׁן	0
אַךְ	0
תַּשְׁבִּיתוּ	0
שְּׂאֹר	0
מִבָּתֵּיכֶם	0
אֹכֵל	0
וְנִכְרְתָה	0
הַנֶּפֶשׁ	0
מִיִּשְׂרָאֵל	0
מִיּוֹם	0
הָרִאשֹׁן	0
הַשְּׁבִעִי	0
מֵעֶרֶב	0
תִשְׁחַט	0
זִבְחִי	2
וַעֲדַיִן	0
קַיָּם	0
מִבְּעֶרֶב	0
חִלֵּק	0
בִּשְׂרִיפָה	0
בַּאֲכִילָה	0
נְבֵלָה	0
תּוֹכִיחַ	0
שֶׁהִיא	0
אֲסוּרָה	0
וְאֵינָהּ	0
הֶפְרֵשׁ	0
בַּהֲנָאָה	0
שׁוֹר	2
הַנִּסְקָל	0
יוֹכִיחַ	0
בִשְׂרִיפָה	0
חַיָּבִין	0
כָּרֵת	0
חֵלֶב	0
שֶׁלַּשּׁוֹר	0
שֶׁחַיָּבִין	0
שֶׁאַף	0
אָדוּן	0
לְשׁוֹנוֹת	0
מֵאַרְבָּעָה	0
וְאָסוּר	0
וְחַיָּבִין	0
וְהַזְּמַן	0
גָּרַם	0
שֶׁאֲסוּרָה	0
אֵינָהּ	0
הַזְּמַן	0